# BIOPS CFT Option Use Case 1: Move one dashboard to another account with assets as bundle API #
Author: Ying Wang, Sr. SA in Gen AI, QuickSight

Created Time: June 2025

## Update pip and boto3

In [ ]:
!pip install --upgrade pip
!pip install --upgrade boto3
get_ipython().system('pip install --upgrade ipynb')

**Only run it one time!!!**

The cell below is to update the bucket policy to allow cross-accounts access: let the targe account to copy the assets in source account.
Please update the template path if you have a new assets bundle to be imported into the target account

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError

# === Configuration ===
bucket_name = ""
source_account_id = ""  # Replace with the AWS Account ID to allow
target_account_id = ""
role_name = ""  # Replace with the role name you want to allow access    

# === Initialize S3 client ===
s3 = boto3.client("s3")

# === Define the new policy statement ===
new_statement1 = {
    "Sid": "AllowAccountAToAccessTemplate",
    "Effect": "Allow",
    "Principal": {
        "AWS": f"arn:aws:iam::{target_account_id}:root"
    },
    "Action": "s3:*",
      "Resource": [
        f"arn:aws:s3:::{bucket_name}",
        f"arn:aws:s3:::{bucket_name}/*"
      ]
}

new_statement2 = {
      "Sid": "AllowCloudFormationAccess",
      "Effect": "Allow",
      "Principal": {
        "AWS": f"arn:aws:iam::{target_account_id}:role/{role_name}"  # Replace role_name with the actual role name
      },
      "Action": "s3:GetObject",
      "Resource": f"arn:aws:s3:::{bucket_name}/*"
    }


# Try to get the existing policy, or create a new one
try:
    response = s3.get_bucket_policy(Bucket=bucket_name)
    policy = json.loads(response['Policy'])
except s3.exceptions.NoSuchBucketPolicy:
    policy = {
        "Version": "2012-10-17",
        "Statement": []
    }

# Always append the new statement (no duplicate check)
policy["Statement"].append(new_statement1)
policy["Statement"].append(new_statement2)

# Update the policy
updated_policy = json.dumps(policy)
s3.put_bucket_policy(Bucket=bucket_name, Policy=updated_policy)
print("Bucket policy updated successfully.")

Bucket policy updated successfully.


## Configuration

In [ ]:
import json
import boto3
import logging
import csv
import io
import os
import tempfile
from typing import Any, Callable, Dict, List, Optional, Union
import sys
import botocore
import requests
import os

#start-Initial set up for the sdk env#
def default_botocore_config() -> botocore.config.Config:
    """Botocore configuration."""
    retries_config: Dict[str, Union[str, int]] = {
        "max_attempts": int(os.getenv("AWS_MAX_ATTEMPTS", "5")),
    }
    mode: Optional[str] = os.getenv("AWS_RETRY_MODE")
    if mode:
        retries_config["mode"] = mode
    return botocore.config.Config(
        retries=retries_config,
        connect_timeout=10,
        max_pool_connections=10,
        user_agent_extra=f"qs_sdk_biops",
    )
sts_client = boto3.client("sts", config=default_botocore_config())
account_id = sts_client.get_caller_identity()["Account"]
qs_client = boto3.client('quicksight')
aws_region = qs_client.meta.region_name
print(aws_region)

us-east-1


In [ ]:
analysis_def=qs_client.describe_analysis_definition(
    AwsAccountId=account_id,
    AnalysisId='',
    
)
from IPython.display import JSON
JSON(analysis_def)

In [ ]:
jobId=''
exp_res = qs_client.start_asset_bundle_export_job(
    AwsAccountId=account_id,
    AssetBundleExportJobId=jobId,
    ResourceArns=[
        'arn:aws:quicksight:us-east-1:<<account-id>>:dashboard/<<dashboard-id>>',
    ],
    IncludeAllDependencies=True,
    IncludePermissions=False,
    ExportFormat='CLOUDFORMATION_JSON')


from IPython.display import JSON
JSON(exp_res)

In [ ]:

des_res = qs_client.describe_asset_bundle_export_job(
    AwsAccountId=account_id,
    AssetBundleExportJobId=jobId
)



from IPython.display import JSON
JSON(des_res)

In [78]:
import time
def WaitForExportToComplete(jobId):
    while True:
        response = qs_client.describe_asset_bundle_export_job(
                    AwsAccountId=account_id,
                    AssetBundleExportJobId=jobId
                    )
        
        job_status = response['JobStatus']
        if job_status in ['SUCCESSFUL', 'FAILED']:
            print(f"Job finished with status: {job_status}")
            return response
        else:
            print(f"Job still running. Current status: {job_status}. Retrying in 10 seconds...")
            time.sleep(10)

In [ ]:
response=WaitForExportToComplete(jobId)

In [ ]:


# === Configuration ===
url = response['DownloadUrl']
local_path = response['ExportFormat'] + '_' + response['AssetBundleExportJobId']
local_file_name = local_path + '.template'
bucket_name = ""  # Replace with your S3 bucket name
s3_key = local_path + '/' + local_file_name

# === Step 1: Download file from URL ===
response = requests.get(url)
if response.status_code == 200:
    with open(local_file_name, "wb") as f:
        f.write(response.content)
else:
    raise Exception(f"Failed to download file. Status code: {response.status_code}")

# === Step 2: Upload file to S3 ===
s3 = boto3.client('s3')
s3.upload_file(local_file_name, bucket_name, s3_key)

print(f"File uploaded to s3://{bucket_name}/{s3_key}")



In [ ]:
s3path = f"{bucket_name}/{s3_key}"
s3url = f"https://{bucket_name}.s3.us-east-1.amazonaws.com/{s3_key}"
print(s3url)


# Validate that s3url is not empty and looks correct
if not s3url or not s3url.startswith("https://"):
    raise ValueError(f"s3url is invalid: {s3url}")

## Now, call cloudformation to create a stack to deploy the quicksight assets

In [ ]:
print(
    "To create the stack in your target account, run the following command in your terminal:\n"
    f"aws cloudformation create-stack "
    f"--stack-name qs-import "
    f"--template-url \"{s3url}\" "
    f"--capabilities CAPABILITY_NAMED_IAM"
)
